<img src="../images/logos/wradlib_logo.svg.png" width=250 alt="wradlib logo png"></img>

# wradlib radar data io, visualisation, gridding and gis export

---

## Overview

Within this notebook, we will cover:

1. Reading radar volume data into xarray based RadarVolume
1. Examination of RadarVolume and Sweeps
1. Plotting of sweeps, simple and mapmaking
1. Gridding and GIS output

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Xarray Basics](https://tutorial.xarray.dev/intro.html) | Helpful | Basic Dataset/DataArray |
| [Matplotlib Basics](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html) | Helpful | Basic Plotting |
| [Cartopy Basics](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | Projections |
| [GDAL Basiscs](https://gdal.org/api/python_bindings.html) | Helpful | Raster |

- **Time to learn**: 15 minutes

---

## Imports

In [ ]:
import glob
import pathlib

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from matplotlib import ticker as tick
from osgeo import gdal

import wradlib as wrl

## Import data into RadarVolume

We have this special case here with Rainbow data where moments are splitted across files. Each file nevertheless consists of all sweeps comprising the volume. We'll use some special nested ordering to read the files.

In [ ]:
fglob = "data/rainbow/meteoswiss/*.vol"

In [ ]:
vol = wrl.io.open_rainbow_mfdataset(fglob, combine="by_coords", concat_dim=None)

### Examine RadarVolume

The RadarVolume is a shallow class which tries to comply to CfRadial2/WMO-FM301, see [WMO-CF_Extensions](https://community.wmo.int/activity-areas/wis/wmo-cf-extensions).

The printout of `RadarVolume` just lists the dimensions and the associated elevations.

In [ ]:
display(vol)

### Root Group

The root-group is essentially an overview over the volume, more or less aligned with CfRadial metadata.

In [ ]:
vol.root

### Sweep Groups

Sweeps are available in a sequence attached to the `RadarVolume` object.

In [ ]:
swp = vol[0]
display(swp)

## Inspect Scan Strategy

Considering volume files it's nice to have an overview over the scan strategy. We can choose some reasonable values for the layout.

In [ ]:
nrays = 360
nbins = 150
range_res = 1000.0
ranges = np.arange(nbins) * range_res
elevs = vol.root.sweep_fixed_angle.values
sitecoords = (
    vol.root.longitude.values.item(),
    vol.root.latitude.values.item(),
    vol.root.altitude.values.item(),
)

beamwidth = 1.0

In [ ]:
ax = wrl.vis.plot_scan_strategy(ranges, elevs, sitecoords)

We can plot it on top of the terrain derived from SRTM DEM.

In [ ]:
import os

os.environ["WRADLIB_EARTHDATA_BEARER_TOKEN"] = ""
os.environ["WRADLIB_DATA"] = "data/wradlib-data"

In [ ]:
ax = wrl.vis.plot_scan_strategy(ranges, elevs, sitecoords, terrain=True)

Let's make the earth go round...

In [ ]:
ax = wrl.vis.plot_scan_strategy(
    ranges, elevs, sitecoords, cg=True, terrain=True, az=180
)

## Plotting Radar Data
### Time vs. Azimuth

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(111)
swp.azimuth.sortby("rtime").plot(x="rtime", marker=".")

### Range vs. Azimuth/Time

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(121)
swp.DBZH.plot(cmap="turbo", ax=ax1)
ax1.set_title(f"{swp.time.values.astype('M8[s]')}")
ax2 = fig.add_subplot(122)
swp.DBZH.sortby("rtime").plot(y="rtime", cmap="turbo", ax=ax2)
ax2.set_title(f"{swp.time.values.astype('M8[s]')}")
plt.tight_layout()

### Georeferenced as Plan Position Indicator

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(111)
swp.DBZH.pipe(wrl.georef.georeference_dataset).plot(
    x="x", y="y", ax=ax1, cmap="turbo", cbar_kwargs=dict(shrink=0.8)
)
ax1.plot(0, 0, "rx", markersize=12)
ax1.set_title(f"{swp.time.values.astype('M8[s]')}")
ax1.grid()
ax1.set_aspect("equal")

### Basic MapMaking with cartopy

The data will be georeferenced as `Azimuthal Equidistant Projection` centered at the radar. For the map projection we will use `Mercator`.

In [ ]:
map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)

In [ ]:
def plot_borders(ax):
    borders = cfeature.NaturalEarthFeature(
        category="cultural", name="admin_0_countries", scale="10m", facecolor="none"
    )
    ax.add_feature(borders, edgecolor="black", lw=2, zorder=4)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
cbar_kwargs = dict(shrink=0.7, pad=0.075)
pm = swp.DBZH.pipe(wrl.georef.georeference_dataset).plot(
    ax=ax, x="x", y="y", cbar_kwargs=cbar_kwargs, cmap="turbo", transform=map_trans
)
plot_borders(ax)
ax.gridlines(draw_labels=True)
ax.plot(
    swp.longitude.values, swp.latitude.values, transform=map_trans, marker="*", c="r"
)
ax.set_title(f"{swp.time.values.astype('M8[s]')}")
ax.set_xlim(-15e4, 45e4)
ax.set_ylim(565e4, 610e4)
plt.tight_layout()

### Plot on curvelinear grid

For Xarray DataArrays wradlib uses a so-called accessor (`wradlib`). To plot on curvelinear grids projection has to be set to `cg`, which uses the matplotlib AXISARTIS namespace.

In [ ]:
fig = plt.figure(figsize=(10, 8))

pm = swp.DBZH.pipe(wrl.georef.georeference_dataset).wradlib.plot(
    proj="cg", fig=fig, cmap="turbo"
)

ax = plt.gca()

# apply eye-candy
caax = ax.parasites[0]
paax = ax.parasites[1]
ax.parasites[1].set_aspect("equal")
t = plt.title(f"{vol[0].time.values.astype('M8[s]')}", y=1.05)
cbar = plt.colorbar(pm, pad=0.075, ax=paax)
caax.set_xlabel("x_range [m]")
caax.set_ylabel("y_range [m]")
plt.text(1.0, 1.05, "azimuth", transform=caax.transAxes, va="bottom", ha="right")
cbar.set_label("reflectivity [dBZ]")

## ODIM_H5 format export and import
### Export to ODIM_H5

In [ ]:
vol.to_odim("test_odim_vol.h5")

### Import from ODIM_H5

In [ ]:
vol2 = wrl.io.open_odim_dataset("test_odim_vol.h5")
display(vol2)

In [ ]:
display(vol2[0])

## Import with xarray backends

We can facilitate the xarray backend's which wradlib provides for the different readers. The xarray backends are capable of loading data into a single Dataset for now. So we need to give some information here too.

### Open single files

The simplest case can only open one file and one group a time!

In [ ]:
ds = xr.open_dataset("test_odim_vol.h5", engine="odim", group="dataset1")
display(ds)

### Open multiple files

Here we just specify the group, which in case of rainbow files is given by the group number.

In [ ]:
ds = xr.open_mfdataset(fglob, engine="rainbow", group=0, combine="by_coords")
display(ds)

## Gridding and Export to GIS formats

- get coordinates from source Dataset with given projection
- calculate target coordinates
- grid using wradlib interpolator
- export to single band geotiff
- use GDAL CLI tools to convert to grayscaled/paletted PNG

In [ ]:
def get_target_grid(ds, nb_pixels):
    xgrid = np.linspace(ds.x.min(), ds.x.max(), nb_pixels, dtype=np.float32)
    ygrid = np.linspace(ds.y.min(), ds.y.max(), nb_pixels, dtype=np.float32)
    grid_xy_raw = np.meshgrid(xgrid, ygrid)
    grid_xy_grid = np.dstack((grid_xy_raw[0], grid_xy_raw[1]))
    return xgrid, ygrid, grid_xy_grid


def get_target_coordinates(grid):
    grid_xy = np.stack((grid[..., 0].ravel(), grid[..., 1].ravel()), axis=-1)
    return grid_xy


def get_source_coordinates(ds):
    xy = np.stack((ds.x.values.ravel(), ds.y.values.ravel()), axis=-1)
    return xy


def coordinates(da, proj, res=100):
    # georeference single sweep
    da = da.pipe(wrl.georef.georeference_dataset, proj=proj)
    # get source coordinates
    src = get_source_coordinates(da)
    # create target grid
    xgrid, ygrid, trg = get_target_grid(da, res)
    return src, trg


def moment_to_gdal(da, trg_grid, driver, ext, path="", proj=None):
    # use wgs84 pseudo mercator if no projection is given
    if proj is None:
        proj = wrl.georef.epsg_to_osr(3857)
    t = da.time.values.astype("M8[s]").astype("O")
    outfilename = f"gridded_{da.name}_{t:%Y%m%d}_{t:%H%M%S}"
    outfilename = os.path.join(path, outfilename)
    f = pathlib.Path(outfilename)
    f.unlink(missing_ok=True)
    res = ip_near(da.values.ravel(), maxdist=1000).reshape(
        (len(trg_grid[0]), len(trg_grid[1]))
    )
    data, xy = wrl.georef.set_raster_origin(res, trg_grid, "upper")
    ds = wrl.georef.create_raster_dataset(data, xy, projection=proj)
    wrl.io.write_raster_dataset(outfilename + ext, ds, driver)

### Coordinates

In [ ]:
%%time
epsg_code = 2056
proj = wrl.georef.epsg_to_osr(epsg_code)
src, trg = coordinates(ds, proj, res=1400)

### Interpolator

In [ ]:
%%time
ip_near = wrl.ipol.Nearest(src, trg.reshape(-1, trg.shape[-1]), remove_missing=7)

### Gridding and Export

In [ ]:
%%time
moment_to_gdal(ds.DBZH, trg, "GTiff", ".tif", proj=proj)

### GDAL info on created GeoTiff

In [ ]:
!gdalinfo gridded_DBZH_20191021_082409.tif

### Translate exported GeoTiff to grayscale PNG

In [ ]:
!gdal_translate -of PNG -ot Byte -scale -30. 60. 0 255 gridded_DBZH_20191021_082409.tif grayscale.png

### Apply colortable to PNG

In [ ]:
with open("colors.txt", "w") as f:
    f.write("0 blue\n")
    f.write("50 yellow\n")
    f.write("100 yellow\n")
    f.write("150 orange\n")
    f.write("200 red\n")
    f.write("250 white\n")

### Display exported PNG's

In [ ]:
!gdaldem color-relief grayscale.png colors.txt paletted.png

<img src="grayscale.png" width=400 alt="grayscale png" align="left"></img>
<img src="paletted.png" width=400 alt="paletted png" align="left"></img>

### Import with Xarray, rasterio backend

In [ ]:
with xr.open_dataset("gridded_DBZH_20191021_082409.tif", engine="rasterio") as ds_grd:
    display(ds_grd)
    ds_grd.band_data.plot(cmap="turbo")

---

## Summary
We've just learned how to use $\omega radlib$'s xarray backends to make radar volume data available as xarray Datasets and DataArrays. Accessing, plotting and exporting data has been shown.

### What's next?
In the next notebook we dive into data quality processing.

## Resources and references

- [xarray](https://docs.xarray.dev)
- [dask](https://docs.dask.org)
- [matplotlib](https://matplotlib.org/stable/index.html)
- [matplotlib axisartist](https://matplotlib.org/stable/tutorials/toolkits/axisartist.html)
- [cartopy](https://scitools.org.uk/cartopy/docs/latest)
- [gdal](https://gdal.org/index.html)
- [wradlib xarray backends](https://docs.wradlib.org/en/stable/notebooks/fileio/wradlib_xarray_backends.html)
- [rioxarray](https://corteva.github.io/rioxarray/stable/)
- [wradlib scan strategy](https://docs.wradlib.org/projects/old-docs/en/1.16.0/notebooks/visualisation/wradlib_plot_scan_strategy.html)
- [Leonardo - Rainbow5](https://electronics.leonardo.com/en/products/rainbow-5-application-software)
- [OPERA ODIM_H5](https://www.eumetnet.eu/activities/observations-programme/current-activities/opera/)
- [WMO JET-OWR](https://community.wmo.int/governance/commission-membership/commission-observation-infrastructure-and-information-systems-infcom/commission-infrastructure-officers/infcom-management-group/standing-committee-measurements-instrumentation-and-traceability-sc-mint/joint-expert-team)